# OpenLCA data extraction script for the bill of materials structure of Xylem


In [94]:
import olca_ipc as ipc
import olca_schema as schema
import pandas as pd
import json

In [95]:
# Pfad zur Excel-Datei
excel_file_path = 'C:\\Users\\ale46522\\OneDrive - Fraunhofer\\Desktop\\Arbeit\\Xylem\\Python Skript - Automatisierung\\Bill_of_Materials.xlsx'

# Excel-Datei in ein DataFrame einlesen
df = pd.read_excel(excel_file_path)

# Sicherstellen, dass die Tabelle mindestens vier und neun Spalten hat
if df.shape[1] >= 9:
    # Daten in einem Dictionary speichern
    data_dict = {}
    for index, row in df.iterrows():
        process_name = row.iloc[3]  # String in der vierten Spalte
        process_category = int(row.iloc[8])  # Integer in der neunten Spalte
        data_dict[process_name] = process_category
else:
    data_dict = {}
    print("Die Tabelle hat weniger als neun Spalten.")

# Dictionary in einen formatierten JSON-String umwandeln
formatted_json = json.dumps(data_dict, indent=4)

# Den formatierten JSON-String ausgeben
# print(formatted_json)

In [96]:
 # Hierarchische Zuordnung der Prozess- und Fluss-IDs
hierarchy_dict = {}
current_parent_ids = {1: None, 2: None, 3: None}

In [97]:
client = ipc.Client(8080)

client

# Zugriff auf einzelne Einträge im Dictionary
for process_name, process_category in data_dict.items():
    # Erstellung der Prozesse 
    process = schema.Process()
    process.name =  process_name
    process.description = f'This process represents the production of {process_name}.'
    process.process_type = schema.ProcessType.UNIT_PROCESS
    process.category = f'Ebene {process_category}'
    process_id = process.id
    

    # Erstellung der Flows
    flow_property_factor = schema.FlowPropertyFactor()
    flow = schema.Flow()
    flow.name = process_name
    flow.category = 'Xylem_flow'
    flow.flow_properties = [flow_property_factor]
    flow.flow_type = schema.FlowType.PRODUCT_FLOW
    flow_id = flow.id

    # Hierarchie speichern
    if process_category == 1:
        current_parent_ids[1] = process_id
        hierarchy_dict[process_id] = {"parent": None, "flow_id": flow_id}
    elif process_category == 2:
        current_parent_ids[2] = process_id
        hierarchy_dict[process_id] = {"parent": current_parent_ids[1], "flow_id": flow_id}
    elif process_category == 3:
        hierarchy_dict[process_id] = {"parent": current_parent_ids[2], "flow_id": flow_id}
   

    # Erstellung des Exchanges
    flow_property = schema.FlowProperty()
    unit = schema.Unit()
    exchange = schema.Exchange()
    exchange.is_input = False
    exchange.flow = flow
    exchange.flow_property = flow_property # Platzhalter
    exchange.unit = unit # Platzhalter
    exchange.amount = 1
    exchange.is_quantitative_reference = True

    # Einfügen der Exchanges in den Prozess
    process.exchanges = [exchange]   
    
    
    # Erstellte Objekte im OpenLCA-Client speichern
    client.put(flow)
    client.put(process)
  

    

   
    # print(f"{process_name} wurde erfolgreich erstellt.")

print(f"Success! Prozesse wurden erfolgreich erstellt.")
print(json.dumps(hierarchy_dict, indent=4))


Success! Prozesse wurden erfolgreich erstellt.
{
    "3809a040-b63f-4ea7-87ab-639bb2cbe8ed": {
        "parent": null,
        "flow_id": "7d9fb9c6-b348-4165-866c-57c9c42d98b0"
    },
    "b127cc0c-e7da-4d9b-bad5-32c10bd6f386": {
        "parent": "3809a040-b63f-4ea7-87ab-639bb2cbe8ed",
        "flow_id": "c80880d2-7f37-4d78-8128-eb3885f25236"
    },
    "99c55504-9eb9-40e3-99c7-94feab958ffc": {
        "parent": "3809a040-b63f-4ea7-87ab-639bb2cbe8ed",
        "flow_id": "99f69e14-5e9c-4279-b7d8-8e0f41f064cc"
    },
    "5b1c44a9-9f81-42e7-a014-8b013add739d": {
        "parent": "3809a040-b63f-4ea7-87ab-639bb2cbe8ed",
        "flow_id": "b18c5772-1103-4881-b846-ff789d1fc397"
    },
    "c879ea38-40e1-4132-878c-7af2b49b8f72": {
        "parent": "3809a040-b63f-4ea7-87ab-639bb2cbe8ed",
        "flow_id": "7472ce81-5dee-49d2-a5f9-c872e3ecd679"
    },
    "65487391-6ccb-48cb-85bc-1ffb9201aefa": {
        "parent": "c879ea38-40e1-4132-878c-7af2b49b8f72",
        "flow_id": "5e4000ed-68e3-4b

# ToDo
- Exchange Erstellung und die Übertragung der erstellten Objekte muss aus der for-Schleife gezogen werden
- neue for-Schleife erstellen und über "hierarchy_dict" iterieren, um korrekte Exchanges mit allen notwendigen Flows auf Input- und Outputseite zu generieren
- Übertragung von Werten mit korrekten Einheiten (number of items, g, mm) implementieren